<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv("https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv")

In [2]:
pd.set_option('display.max_columns', None)

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.shape

(7043, 21)

In [4]:
df.isnull().any()

customerID          False
gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
OnlineBackup        False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges        False
Churn               False
dtype: bool

In [5]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,6705-LNMDD,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [6]:
df = df.drop(['customerID'], axis=1)
df['TotalCharges'] = df['TotalCharges'].str.strip()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
df['TotalCharges'].fillna(0, inplace=True)

In [7]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [8]:
df.isnull().sum()


gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df['Churn'], test_size=0.25, random_state=1337, stratify=df['Churn'])

In [10]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

encoder = ce.OneHotEncoder()

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [11]:
X_train.head()

,gender_1,gender_2,SeniorCitizen,Partner_1,Partner_2,Dependents_1,Dependents_2,tenure,PhoneService_1,PhoneService_2,MultipleLines_1,MultipleLines_2,MultipleLines_3,InternetService_1,InternetService_2,InternetService_3,OnlineSecurity_1,OnlineSecurity_2,OnlineSecurity_3,OnlineBackup_1,OnlineBackup_2,OnlineBackup_3,DeviceProtection_1,DeviceProtection_2,DeviceProtection_3,TechSupport_1,TechSupport_2,TechSupport_3,StreamingTV_1,StreamingTV_2,StreamingTV_3,StreamingMovies_1,StreamingMovies_2,StreamingMovies_3,Contract_1,Contract_2,Contract_3,PaperlessBilling_1,PaperlessBilling_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,PaymentMethod_4,MonthlyCharges,TotalCharges
3388,1,0,0,1,0,1,0,65,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,55.15,3673.15
4731,0,1,0,1,0,0,1,19,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,19.90,357.70
1021,0,1,0,1,0,1,0,28,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,54.35,1426.45
6968,1,0,1,0,1,1,0,22,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,74.40,1692.60
6361,1,0,0,0,1,1,0,7,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,62.80,418.30


In [12]:
X_train.values.shape


(5282, 45)

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Trying logistic regression for baseline.

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

lr.score(X_test, y_test)


0.8103350369108461

In [ ]:
import tensorflow as tf
from tensorflow import keras


In [17]:
df['Churn'].value_counts(normalize=True)


No     0.73463
Yes    0.26537
Name: Churn, dtype: float64

In [19]:
from kerastuner import HyperModel

class MultilayerPerceptron(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        model.add(Dense(units=hp.Int('units', min_value=16, max_value=512, step=16, default=128), activation='sigmoid'))
        model.add(Dense(units=hp.Int('units', min_value=16, max_value=512, step=16, default=128), activation='sigmoid'))
        model.add(Dense(self.num_classes, activation='sigmoid'))

        model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2,
                                                               sampling='LOG', default=1e-3)),
                      loss='BinaryCrossentropy', metrics=['accuracy'])

        return model

In [20]:
from kerastuner.tuners import RandomSearch

tf.keras.backend.set_floatx('float64') # resolves some warnings

INPUT_SHAPE = (5282, 45)
NUM_CLASSES = 1

hypermodel = MultilayerPerceptron(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(hypermodel, objective='val_accuracy', seed=1337, max_trials=40, executions_per_trial=3,
                     directory='random_search')

In [21]:
tuner.search_space_summary()

In [22]:
from tensorflow.keras.callbacks import EarlyStopping


stop = EarlyStopping(monitor='val_accuracy', patience=5)
# tuner.search(X_train, y_train, epochs=20, validation_split=0.1, verbose=0, callbacks=[stop])
tuner.search(X_train, y_train, epochs=9999, validation_data=(X_test, y_test), verbose=0, callbacks=[stop])

INFO:tensorflow:Oracle triggered exit


In [23]:
tuner.results_summary()

In [24]:
best_model = tuner.get_best_models(num_models=1)[0]

loss, accuracy = best_model.evaluate(X_test, y_test)

56/56 [==============================] - ETA: 0s - loss: 0.4608 - accuracy: 0.7812 - ETA: 0s - loss: 0.4155 - accuracy: 0.8125 - 0s 1ms/step - loss: 0.4149 - accuracy: 0.8149


In [25]:
print(accuracy)

0.814877910278251


## Just putting gridsearch here to say I did it. In practice I very rarely would ever see myself using grid search vs a vs a bayesian/random alternative with NNs.

In [29]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
np.random.seed(1337)


# Function to create model, required for KerasClassifier
def create_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, activation='sigmoid', input_dim=(45)))
    model.add(Dense(neurons, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [5, 10, 15],
              'neurons': [32, 64, 128],
              'epochs': [15]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7987561279779822 using {'batch_size': 15, 'epochs': 15, 'neurons': 128}
Means: 0.7964834007052549, Stdev: 0.012608280683487462 with: {'batch_size': 5, 'epochs': 15, 'neurons': 32}
Means: 0.793642491614346, Stdev: 0.012103904165070107 with: {'batch_size': 5, 'epochs': 15, 'neurons': 64}
Means: 0.7942110317938132, Stdev: 0.014011961125859227 with: {'batch_size': 5, 'epochs': 15, 'neurons': 128}
Means: 0.7919411714113701, Stdev: 0.01851219210681091 with: {'batch_size': 10, 'epochs': 15, 'neurons': 32}
Means: 0.7902351925116826, Stdev: 0.015189760223073676 with: {'batch_size': 10, 'epochs': 15, 'neurons': 64}
Means: 0.7930764599638771, Stdev: 0.016967140084645876 with: {'batch_size': 10, 'epochs': 15, 'neurons': 128}
Means: 0.7972424099079728, Stdev: 0.017372238427831962 with: {'batch_size': 15, 'epochs': 15, 'neurons': 32}
Means: 0.7966740489091484, Stdev: 0.01727341666183862 with: {'batch_size': 15, 'epochs': 15, 'neurons': 64}
Means: 0.7987561279779822, Stdev: 0.0156375097517162

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?